In [31]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [32]:
# Define column names
columns = [
    "unit_ID",
    "cycles",
    "setting_1",
    "setting_2",
    "setting_3",
    "T2",
    "T24",
    "T30",
    "T50",
    "P2",
    "P15",
    "P30",
    "Nf",
    "Nc",
    "epr",
    "Ps30",
    "phi",
    "NRf",
    "NRc",
    "BPR",
    "farB",
    "htBleed",
    "Nf_dmd",
    "PCNfR_dmd",
    "W31",
    "W32",
]

truth_columns = [
    "unit_ID",
    "RUL",
]

In [33]:
#File paths
train_path = "../CMAPSS_GRP_2/dataset/train_FD001.txt"
test_path = "../CMAPSS_GRP_2/dataset/test_FD001.txt"
rul_path = "../CMAPSS_GRP_2/dataset/RUL_FD001.txt"

# Data Preprocessing
- Dropping unnecessary columns
- Normalization (Min Max Scaling)
- Labelling of Data

In [34]:
# Function to (a) load data from file paths, (b) drop NaN columns and (c) rename columns.
def load_and_clean_data(train_path, test_path, rul_path, columns, truth_columns):
    
    # Load datasets
    train_df = pd.read_csv(train_path, sep=" ", header=None)
    test_df = pd.read_csv(test_path, sep=" ", header=None)
    rul_df = pd.read_csv(rul_path, sep=" ", header=None)

    # Drop NaN columns and rename columns,there exists 2 NaN columns after the 26th data column
    train_df.dropna(axis=1, inplace=True)
    test_df.dropna(axis=1, inplace=True)
    rul_df.dropna(axis=1, inplace=True)
    rul_df.insert(0, "unit_ID", range(1, len(rul_df) + 1))

    train_df.columns = columns
    test_df.columns = columns
    rul_df.columns = truth_columns

    return train_df, test_df, rul_df

# 1. Load and clean data
train_df, test_df, rul_df = load_and_clean_data(train_path, test_path, rul_path, columns, truth_columns)

In [35]:
# 2. Feature Selection: Remove redundant columns based on observations from sensor measurement plots
columns_to_remove = ["T2", "P2", "P15", "epr", "farB", "Nf_dmd", "PCNfR_dmd"]
train_df.drop(columns=columns_to_remove, inplace=True)
test_df.drop(columns=columns_to_remove, inplace=True)

# New Training Data Generation - Preprocessing
- Creating a new training data from existing training data using sliding window

In [36]:
# 3. Parameters for sliding windows,shifts,early RUL(piece-wise method) and empty lists to store generated sample data from using shifting window
window_length = 30
shift = 1
early_rul = 125           
processed_train_data = []
processed_train_targets = []
num_test_windows = 5     
processed_test_data = []
num_test_windows_list = []

train_data_first_column = train_df["unit_ID"]
test_data_first_column = test_df["unit_ID"]

# Data Normalisation
- Carry out data normalisation on the newly generated training data

In [37]:
# 4. Perform normalisation with a desired range between 0 and 1.

# Initialize the MinMaxScaler with a desired range
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit and transform the data
train_df = scaler.fit_transform(train_df.drop(columns=['unit_ID']))
test_df = scaler.transform(test_df.drop(columns=['unit_ID']))

train_df = pd.DataFrame(data = np.c_[train_data_first_column, train_df])
test_df = pd.DataFrame(data = np.c_[test_data_first_column, test_df])

num_train_machines = len(train_df[0].unique())
num_test_machines = len(test_df[0].unique())

In [38]:
# # Normalization
# from sklearn.preprocessing import MinMaxScaler

# # Initialize the scaler
# scaler = MinMaxScaler()

# # Separate the columns to normalize and the columns to skip
# columns_to_skip = train_df_dropped.columns[:2]
# columns_to_normalize = train_df_dropped.columns[2:]

# # Normalize only the selected columns
# normalized_data = scaler.fit_transform(train_df_dropped[columns_to_normalize])

# # Combine the normalized and unnormalized columns
# train_df_normalized = pd.DataFrame(train_df_dropped[columns_to_skip].values, columns=columns_to_skip)
# train_df_normalized = pd.concat([train_df_normalized, pd.DataFrame(normalized_data, columns=columns_to_normalize)], axis=1)

# # Display the normalized DataFrame
# print("Normalized Data (0-1 range):")
# print(train_df_normalized.head())

# Data Labelling
- Labelling of RUL Data using the Piecewise method.

In [39]:
# Function to label RUL data using the Piecewise methhod.
def process_targets(data_length, early_rul = None):
    if early_rul == None:
        return np.arange(data_length-1, -1, -1)
    else:
        early_rul_duration = data_length - early_rul
        if early_rul_duration <= 0:
            
            # Return decreasing RUL based on length of train data if RUL is lesser than specified early RUL (of 125)
            return np.arange(data_length-1, -1, -1)
        else:
            # Return a RUL of constant value (of 125) until it reaches the point when RUL starts to drop below 125.
            return np.append(early_rul*np.ones(shape = (early_rul_duration,)), np.arange(early_rul-1, -1, -1))

In [40]:
# Function to generate a new set of training data from existing training data using shifting window method.
def process_input_data_with_targets(input_data, target_data = None, window_length = 1, shift = 1):
    num_batches = int(np.floor((len(input_data) - window_length)/shift)) + 1
    num_features = input_data.shape[1]
    output_data = np.repeat(np.nan, repeats = num_batches * window_length * num_features).reshape(num_batches, window_length,
                                                                                                  num_features)
    if target_data is None:
        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
        return output_data
    else:
        output_targets = np.repeat(np.nan, repeats = num_batches)
        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
            output_targets[batch] = target_data[(shift*batch + (window_length-1))]
        return output_data, output_targets

In [41]:
# Actual generation a new set of training data from existing training data using shifting window method.
for i in np.arange(1, num_train_machines + 1):
    
    # Temporarily getting train data that belongs to a certain engine number and dropping its engine ID column
    temp_train_data = train_df[train_df[0] == i].drop(columns = [0]).values
    
    # Determine whether it is possible to extract training data with the specified window length.
    if (len(temp_train_data) < window_length):
        print("Train engine {} doesn't have enough data for window_length of {}".format(i, window_length))
        raise AssertionError("Window length is larger than number of data points for some engines. "
                             "Try decreasing window length.")
    
    # Generating a RUL target column for every entry in the temporarily train data
    temp_train_targets = process_targets(data_length = temp_train_data.shape[0], early_rul = early_rul)
    
    # Generating a new sample using sliding window for every engine.
    data_for_a_machine, targets_for_a_machine = process_input_data_with_targets(temp_train_data, temp_train_targets, 
                                                                                window_length= window_length, shift = shift)
    # Appending the aggregated train data for every engine to a list.
    processed_train_data.append(data_for_a_machine)
    
    # Appending the aggregated RUL target data for every engine to a list.
    processed_train_targets.append(targets_for_a_machine)

# Compiling the newly generated data (for train and RUL) from sliding window into a dataframe.
processed_train_data = np.concatenate(processed_train_data)
processed_train_targets = np.concatenate(processed_train_targets)

In [42]:
# Generate test data

# Type here

In [43]:
# Labelling of Data with RUL and Piecewise
# # 1) Labelling RUL
# train_df_normalized['RUL'] = train_df_normalized.groupby('engine_id')['cycle'].transform(lambda x: x.max() - x)

# # 2) Labelling PWRUL
# # Set the early RUL threshold
# early_rul_threshold = 120

# # Define the piecewise linear degradation function
# def piecewise_rul(cycle, max_cycle):
#     remaining_life = max_cycle - cycle
#     if remaining_life > early_rul_threshold:
#         return early_rul_threshold  # slower degradation in the early phase
#     else:
#         return remaining_life  # direct linear degradation after threshold
    
# train_df_normalized["PWRUL"] = train_df_normalized.apply(lambda row: piecewise_rul(row['cycle'], row['cycle'] + row['RUL']), axis=1)


# Training of Model

## Linear Regression

In [44]:
# Linera Regresion

In [45]:
# # Data Preparation for LSTM/Bi-lstm/Cnn-lstm
# # Define sequence length
# sequence_length = 30

# # Identify feature columns
# feature_columns = [col for col in train_df_normalized.columns if col not in ['engine_id', 'cycle', 'RUL', 'PWRUL']]

# # Initialize lists for sequences and labels
# X = []
# y = []

# # Generate sequences and labels
# for engine_id in train_df_normalized['engine_id'].unique():
#     engine_data = train_df_normalized[train_df_normalized['engine_id'] == engine_id].reset_index(drop=True)
#     for i in range(sequence_length, len(engine_data)):
#         # Extract sequence of sensor readings
#         seq_x = engine_data[feature_columns].iloc[i-sequence_length:i].values
#         # Extract the RUL value at the end of the sequence
#         seq_y = engine_data['PWRUL'].iloc[i]
#         X.append(seq_x)
#         y.append(seq_y)

# # Convert to NumPy arrays
# X = np.array(X)
# y = np.array(y)

# print("Input shape:", X.shape)
# print("Labels shape:", y.shape)

# Data Shuffling

In [46]:
true_rul = rul_df["RUL"].values

# Shuffle training data
index = np.random.permutation(len(processed_train_targets))
processed_train_data, processed_train_targets = processed_train_data[index], processed_train_targets[index]

# Perform splitting of training data into training and validation dataset.

In [47]:
from sklearn.model_selection import train_test_split

processed_train_data, processed_val_data, processed_train_targets, processed_val_targets = train_test_split(processed_train_data,
                                                                                                            processed_train_targets,
                                                                                                            test_size = 0.2,
                                                                                                            random_state = 83)

In [48]:
print("Processed train data shape: ", processed_train_data.shape)
print("Processed validation data shape: ", processed_val_data.shape)
print("Processed train targets shape: ", processed_train_targets.shape)
print("Processed validation targets shape: ", processed_val_targets.shape)

Processed train data shape:  (14184, 30, 18)
Processed validation data shape:  (3547, 30, 18)
Processed train targets shape:  (14184,)
Processed validation targets shape:  (3547,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Bidirectional, Dropout

# LSTM

In [ ]:
def create_compiled_model_LSTM():
    model = Sequential([
        # CNN layers for spatial feature extraction
        layers.Conv1D(64, kernel_size=3, activation="relu", input_shape=(window_length, 18)),
        layers.Conv1D(64, kernel_size=3, activation="relu"),
        layers.MaxPooling1D(pool_size=2),  # Downsampling to reduce the sequence length
        layers.Dropout(0.2),

        # LSTM layers for temporal pattern recognition
        layers.LSTM(128, return_sequences=True, activation="tanh"),
        layers.Dropout(0.2),
        layers.LSTM(64, activation="tanh", return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(32, activation="tanh"),
        layers.Dropout(0.2),

        # Fully connected layers
        layers.Dense(96, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(128, activation="relu"),
        layers.BatchNormalization(),
        
        # Output layer
        layers.Dense(1)
    ])

    model.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
    )
    
    return model

model_LSTM = create_compiled_model_LSTM()
history = model_LSTM.fit(processed_train_data, processed_train_targets, epochs = 5,
                    validation_data = (processed_val_data, processed_val_targets),
                    batch_size = 128, verbose = 2)

## Bi-LSTM


In [ ]:
def create_compiled_model_BiLSTM():
    model = Sequential([
        layers.Bidirectional(layers.LSTM(128, return_sequences=True, activation="tanh"), input_shape=(window_length, 18)),
        layers.Bidirectional(layers.LSTM(64, return_sequences=True, activation="tanh")),
        layers.Bidirectional(layers.LSTM(32, activation="tanh")),
        layers.Dense(96, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(loss="mse", optimizer= tf.keras.optimizers.Adam(learning_rate=0.001))
    return model

model_BiLSTM = create_compiled_model_BiLSTM()
history = model_BiLSTM.fit(processed_train_data, processed_train_targets, epochs = 5,
                    validation_data = (processed_val_data, processed_val_targets),
                    batch_size = 128, verbose = 2)

# CNN-LSTM

In [ ]:
#Alternative LSTM
def create_compiled_model_CNNLSTM():
    model = Sequential([
        # CNN layers for spatial feature extraction
        layers.Conv1D(64, kernel_size=3, activation="relu", input_shape=(window_length, 18)),
        layers.Conv1D(64, kernel_size=3, activation="relu"),
        layers.MaxPooling1D(pool_size=2),  # Downsampling to reduce the sequence length
        layers.Dropout(0.2),

        # LSTM layers for temporal pattern recognition
        layers.LSTM(128, return_sequences=True, activation="tanh"),
        layers.Dropout(0.2),
        layers.LSTM(64, activation="tanh", return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(32, activation="tanh"),
        layers.Dropout(0.2),

        # Fully connected layers
        layers.Dense(96, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(128, activation="relu"),
        layers.BatchNormalization(),
        
        # Output layer
        layers.Dense(1)
    ])

    model.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
    )
    
    return model

model_CNNLSTM = create_compiled_model_CNNLSTM()
history = model_CNNLSTM.fit(processed_train_data, processed_train_targets, epochs = 5,
                    validation_data = (processed_val_data, processed_val_targets),
                    batch_size = 128, verbose = 2)

In [ ]:

# Define the model (CNN-LSTM)
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, len(feature_columns))))
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=2))
cnn_lstm_model.add(LSTM(units=100))
cnn_lstm_model.add(Dense(units=1))

# Compile the model
cnn_lstm_model.compile(optimizer='adam', loss='mean_squared_error')

print(cnn_lstm_model.summary())

# Training the Model
history = cnn_lstm_model.fit(X, y, 
                    epochs=50, 
                    batch_size=64)

# 

# Testing Models on Test Data
- LSTM
- Bi-LSTM
- CNN+LSTM

In [ ]:
# Prepping test data
X_test = []
y_test = []

for engine_id in test_df_normalized['engine_id'].unique():
    engine_data = test_df_normalized[test_df_normalized['engine_id'] == engine_id].reset_index(drop=True)
    if len(engine_data) >= sequence_length:
        # Use only the last sequence
        seq_x = engine_data[feature_columns].iloc[-sequence_length:].values
        X_test.append(seq_x)
        # Get the true RUL for this engine
        seq_y = true_rul.loc[engine_id - 1].values[0]
        y_test.append(seq_y)
    else:
        print(f"Engine {engine_id} has insufficient data for the defined sequence length.")

X_test = np.array(X_test)
y_test = np.array(y_test)

# Testing Prediction

## Linear Regresssion

## LSTM

## Bi-LSTM

## CNN-LSTM

In [ ]:
from sklearn.metrics import mean_squared_error

# Predict RUL on test data
y_test_pred = cnn_lstm_model.predict(X_test)

# Evaluate
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test Mean Squared Error:", test_mse)

# Comparing the Models